# Text Generation with RNNs


In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import random

### 1 Dataset
Define the path of the file, you want to read and train the model on


In [2]:
import urllib  # the lib that handles the url stuff

text = ""

for line in urllib.request.urlopen("https://raw.githubusercontent.com/GITenberg/Moby-Dick--Or-The-Whale_2701/master/2701.txt"):
    text += line.decode('utf-8')


#### Inspect the dataset
Take a look at the first 250 characters in text

In [3]:
print(text[:250])

The Project Gutenberg EBook of Moby Dick; or The Whale, by Herman Melville

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Pro


In [4]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

86 unique characters


### 2 Process the dataset for the learning task
The task that we want our model to achieve is: given a character, or a sequence of characters, what is the most probable next character?

To achieve this, we will input a sequence of characters to the model, and train the model to predict the output, that is, the following character at each time step. RNNs maintain an internal state that depends on previously seen elements, so information about all characters seen up until a given moment will be taken into account in generating the prediction.

#### Vectorize the text
Before we begin training our RNN model, we'll need to create a numerical representation of our text-based dataset. To do this, we'll generate two lookup tables: one that maps characters to numbers, and a second that maps numbers back to characters. Recall that we just identified the unique characters present in the text.

In [5]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
text_as_int = np.array([char2idx[c] for c in text])

# Create a mapping from indices to characters
idx2char = np.array(vocab)


This gives us an integer representation for each character. Observe that the unique characters (i.e., our vocabulary) in the text are mapped as indices from 0 to len(unique). Let's take a peek at this numerical representation of our dataset:

In [6]:

print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '#' :   5,
  '$' :   6,
  '%' :   7,
  '&' :   8,
  "'" :   9,
  '(' :  10,
  ')' :  11,
  '*' :  12,
  ',' :  13,
  '-' :  14,
  '.' :  15,
  '/' :  16,
  '0' :  17,
  '1' :  18,
  '2' :  19,
  ...
}




We can also look at how the first part of the text is mapped to an integer representation:

In [7]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))



'The Project G' ---- characters mapped to int ---- > [50 67 64  2 46 77 74 69 64 62 79  2 37]


#### Defining a method to encode one hot labels

In [8]:
def one_hot_encode(arr, n_labels):
    one_hot = np.zeros((len(arr), n_labels))
    for i, el in enumerate(arr):
        one_hot[i][el] = 1
    return one_hot


#### Defining a method to make mini-batches for training

In [9]:
def get_batches(arr, batch_size, seq_length, vocab_len):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.

       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    x_batch = []
    y_batch = []
    start_idxs = list(range(len(arr) - 10 * seq_length))
    random.shuffle(start_idxs)
    for start_idx in start_idxs:
        chunk = arr[start_idx:start_idx + seq_length]
        x = one_hot_encode(chunk[:-1], vocab_len)
        y = chunk[-1]
        x_batch.append(x)
        y_batch.append(y)
        if len(x_batch) == batch_size:
            yield torch.FloatTensor(x_batch), torch.LongTensor(y_batch)
            x_batch = []
            y_batch = []
    



## 3 The Recurrent Neural Network (RNN) model


###### Check if GPU is available

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda



### Declaring the model

In [11]:
class VanillaRNN(nn.Module):
    def __init__(self, x_size, a_size, y_size):
        super().__init__()
        # Either w_ax or w_aa doesn't need a bias
        self.w_ax = nn.Linear(x_size, a_size)
        self.w_aa = nn.Linear(a_size, a_size, bias=False)
        self.w_ya = nn.Linear(a_size, y_size)
        self.a_size = a_size
        
    def forward(self, x, last_a):
        '''Forward pass through the network
        x is the input and `hidden` is the hidden/cell state .'''
        a = nn.functional.relu(self.w_aa(last_a) + self.w_ax(x))
        out = nn.functional.softmax(self.w_ya(a), dim=1)
        return out, a

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        a = torch.zeros((batch_size, self.a_size))
        return a

In [12]:
class LSTMCharRNN(nn.Module):
    def __init__(self, y_size, a_size, x_size, n_layers=2,
                 drop_prob = 0.5):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        '''TODO: define the layers you need for the model'''


    def forward(self, x, hidden):
        '''TODO: Forward pass through the network
        x is the input and `hidden` is the hidden/cell state .'''

        # return the final output and the hidden state
        return out, hidden_t

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        
        #hidden = # TODO

        return hidden


#### Declaring the train method

In [13]:
def train(model, data, vocab_len, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network

        Arguments
        ---------

        model: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    '''
    model.train()

    opt = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # create training and validation data
    val_idx = int(len(data) * (1 - val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    train_losses = []
    val_losses = []
    for e in range(epochs):
        running_train_losses = []
        running_val_losses = []
        h = model.init_hidden(batch_size)

        for x, y in tqdm(get_batches(data, batch_size, seq_length, vocab_len)):
            # Only process on full batches
            if x.shape[0] != batch_size:
                continue
            opt.zero_grad()
            a = model.init_hidden(batch_size).to(device)
            x, y = x.to(device), y.to(device)
            for char_pos in range(x.shape[1]):
                inp = x[:, char_pos, :]
                out, a = model(inp, a)
            
            loss = criterion(out, y)
            loss.backward()
            opt.step()
            running_train_losses.append(loss.item())
        
        with torch.no_grad():
            for x, y in tqdm(get_batches(val_data, batch_size, seq_length, vocab_len)):
                if x.shape[0] != batch_size:
                    continue
                a = model.init_hidden(batch_size).to(device)
                x, y = x.to(device), y.to(device)
                for char_pos in range(x.shape[1]):
                    inp = x[:, char_pos, :]
                    out, a = model(inp, a)

                loss = criterion(out, y)
                running_val_losses.append(loss.item())
                
        train_loss = np.mean(np.array(running_train_losses))
        val_loss = np.mean(np.array(running_val_losses))
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        
        print(f"Epoch {e + 1}/{epochs} => train_loss: {train_loss} val_loss: {val_loss}")
    
    return train_losses, val_losses


##### Defining a method to generate the next character

In [14]:
def predict(model, char, vocab_len, h=None):
    ''' Given a character, predict the next character.
        Returns the predicted character and the hidden state.
    '''

    # tensor inputs
    x = np.array([[char2idx[char]]])
    x = one_hot_encode(x, len(model.vocab))
    inputs = torch.from_numpy(x)

    if (train_on_gpu):
        inputs = inputs.cuda()

    # detach hidden state from history
    h = tuple([each.data for each in h])
    '''TODO: feed the current input into the model and generate output'''
    output, h = model(inputs, h) # TODO

    # get the character probabilities
    p = F.softmax(out, dim=1).data.to(device)

    # get top characters
    if top_k is None:
        top_ch = np.arange(len(model.vocab))
    else:
        p, top_ch = p.topk(top_k)
        top_ch = top_ch.numpy().squeeze()

    # select the likely next character with some element of randomness
    p = p.numpy().squeeze()
    char = np.random.choice(top_ch, p=p / p.sum())

    # return the encoded value of the predicted char and the hidden state
    return idx2char[char], h



#### Declaring a method to generate new text

In [15]:
def sample(model, size, prime='The', top_k=None):
    if (train_on_gpu):
        model.cuda()
    else:
        model.cpu()

    model.eval()  # eval mode

    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = model.init_hidden(1)
    for ch in prime:
        char, h = predict(model, ch, h, top_k=top_k)

    chars.append(char)

    for ii in range(size):
        '''TODO: pass in the previous character and get a new one'''
        # char, h = # TODO
        chars.append(char)

    model.train()
    return ''.join(chars)


#### Generate new Text using the RNN model

###### Define and print the net

In [16]:
vanilla_model = VanillaRNN(x_size=len(vocab), a_size=128, y_size=len(vocab)).to(device)
print(vanilla_model)
"""lstm_model = LSTMCharRNN(vocab, n_hidden, n_layers)
print(lstm_model)"""

VanillaRNN(
  (w_ax): Linear(in_features=86, out_features=128, bias=True)
  (w_aa): Linear(in_features=128, out_features=128, bias=False)
  (w_ya): Linear(in_features=128, out_features=86, bias=True)
)


'lstm_model = LSTMCharRNN(vocab, n_hidden, n_layers)\nprint(lstm_model)'


###### Declaring the hyperparameters

In [17]:
''''TODO: Try changing the hyperparameters in the network to see how it affects performance'''
batch_size = 128
seq_length = 20
n_epochs = 3


##### Train the model and have fun with the generated texts

In [ ]:
van_train_losses, van_val_losses = train(vanilla_model, text_as_int, vocab_len=len(vocab), epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

8840it [00:51, 173.04it/s]
982it [00:03, 291.14it/s]
0it [00:00, ?it/s]

Epoch 1/3 => train_loss: 4.168095596775211 val_loss: 4.148660471871525


8840it [00:50, 176.17it/s]
982it [00:03, 300.54it/s]
0it [00:00, ?it/s]

Epoch 2/3 => train_loss: 4.140311367074828 val_loss: 4.145475460895453


1515it [00:09, 175.18it/s]

In [ ]:
train(lstm_model, text_as_int, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)